# Feature Engineering

#### Steps:
1. Text Cleaning
2. Category Labeling
3. Train-test split
4. TD-IDF text reprezentation

In [1]:
import pickle
import pandas as pd
from stop_words import safe_get_stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [2]:
df_path = "../data/data2.csv"
df = pd.read_csv(df_path)
df.head()

,topic,title,text,author,date,visitors,rate,vote_count,key_words
0,Крими,Нова измама: Точат ни банковите карти,Нова измама с източване на кредитни карти върл...,Мария Атанасова,2020-10-19 08:10,26 138,2.4,44.0,"['банкови', 'карти', 'банка', 'измама']"
1,Култура,Почина братът на Карлос Сантана,"Китаристът Хорхе Сантана, по-младият брат на К...",Венелина Маринова,2020-05-19 10:05,1 538,4.5,2.0,"['карлос', 'сантана', 'брат', 'почина', 'хорхе..."
2,Авто,Над 1000 коня за следващото BMW M5,Новото поколение на BMW M5 ще има изцяло елект...,Георги Касабов,2020-06-01 10:06,4 087,3.7,9.0,"['bmw', 'm5', '5er', 'g60']"
3,Любопитно,Владо Карамазов пребори коварна диагноза,Театралната и кинозвезда Владимир Карамазов пр...,Събина Андреева,2021-08-20 09:08,8 896,3.5,17.0,"['владо', 'карамазов', 'диагноза', 'болест', '..."
4,Крими,"Постоянен арест за дрогирания тираджия, предиз...",Окръжен съд - Бургас остави за постоянно в аре...,Мария Атанасова,2020-09-14 18:09,3 518,NaN,0.0,['айтос']


# Text cleaning

###  Special character cleaning

In [3]:

# \r and \n
df['text'] = df['text'].str.replace("\r", " ")
df['text'] = df['text'].str.replace("\n", " ")

# space
df['text'] = df['text'].str.replace("    ", " ")

# quote
df['text'] = df['text'].str.replace('"', '')

### Make all lower case

In [4]:
df['text'] = df['text'].str.lower()

### Remove punctuation 

In [5]:
punctuation_signs = list(["?", ":", "!", ".", ",", ";", "„", "”"])

for punctuation_sign in punctuation_signs:
    df['text'] = df['text'].str.replace(punctuation_sign, '', regex=False)

In [6]:
df.iloc[0]['text']

'нова измама с източване на кредитни карти върлува из най-голямата платформа за краткосрочно отдаване на имоти airnbnb пише в трудсхемата е изключително проста но ако се вържат на нея собствениците на апартаменти и къщи ще носят наказателна отговорност за разлика от самите измамници предупреждават експерти  всъщност всичко започва с мейл до съответния собственик в който се разказва една и съща история с малка разлика в детайлите  в писмото кандидатът за наемане на съответния имот предлага да резервира собствеността и да преведе пари за ползването ѝ за определен периодно да не ползва реално апартамента или къщата и в резултат на това да получи превод със 75 на сто от предплатените пари обратнов писмата често се гарантира всеки месец така да се плащат 10 000 еврочистата печалба за собственика на имота е 2500 евро като при това той може да отдава и недвижимостите и на други клиенти обясниха собственици на имоти според обясненията на измамниците причината за превъртането на тези пари е че 

In [7]:
digits = list(range(10))

for digit in digits:
    df['text'] = df['text'].str.replace(str(digit), '', regex=False)

### Stop words removal

In [8]:
stop_words = safe_get_stop_words("bulgarian")
# len(stop_words)
stop_words[0:10]

['а', 'автентичен', 'аз', 'ако', 'ала', 'бе', 'без', 'беше', 'би', 'бивш']

In [9]:
for stop_word in stop_words:
    stop_word =  " " + stop_word + " "
    df['text'] = df['text'].str.replace(stop_word, ' ')

c:\users\georgivankov\desktop\ai\text mining\venv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
df.iloc[0]['text']

'нова измама източване кредитни карти върлува най-голямата платформа краткосрочно отдаване имоти airnbnb пише трудсхемата изключително проста вържат собствениците апартаменти къщи носят наказателна отговорност разлика самите измамници предупреждават експерти  всъщност започва мейл съответния собственик разказва съща история малка разлика детайлите  писмото кандидатът наемане съответния имот предлага резервира собствеността преведе пари ползването ѝ определен периодно ползва реално апартамента къщата резултат получи превод  сто предплатените пари обратнов писмата гарантира плащат   еврочистата печалба собственика имота  евро отдава недвижимостите клиенти обясниха собственици имоти обясненията измамниците причината превъртането пари всъщност сумата платена работодателя наемащия можел отседне приятели спести сумата същото плащайки наем airbnb получи документ отчете работодателя сиреално изпират пари крадени кредитни карти алармираха експерти'

## Topic Labeling

In [11]:
topic_codes = {
    "Бизнес": 0,
    "Имоти": 1,
    "Технологии": 2,
    "Култура": 3,
    'Спорт': 4,
    'Любопитно': 5,
    "Крими": 6,
    "Авто": 7,
    "topic":1
}
# Topic mapping
df['topic_code'] = df['topic']
df = df.replace({'topic_code':topic_codes})

In [12]:
df.head()

,topic,title,text,author,date,visitors,rate,vote_count,key_words,topic_code
0,Крими,Нова измама: Точат ни банковите карти,нова измама източване кредитни карти върлува н...,Мария Атанасова,2020-10-19 08:10,26 138,2.4,44.0,"['банкови', 'карти', 'банка', 'измама']",6
1,Култура,Почина братът на Карлос Сантана,китаристът хорхе сантана по-младият брат карло...,Венелина Маринова,2020-05-19 10:05,1 538,4.5,2.0,"['карлос', 'сантана', 'брат', 'почина', 'хорхе...",3
2,Авто,Над 1000 коня за следващото BMW M5,новото поколение bmw m изцяло електрическа вер...,Георги Касабов,2020-06-01 10:06,4 087,3.7,9.0,"['bmw', 'm5', '5er', 'g60']",7
3,Любопитно,Владо Карамазов пребори коварна диагноза,театралната кинозвезда владимир карамазов преб...,Събина Андреева,2021-08-20 09:08,8 896,3.5,17.0,"['владо', 'карамазов', 'диагноза', 'болест', '...",5
4,Крими,"Постоянен арест за дрогирания тираджия, предиз...",окръжен съд - бургас остави постоянно ареста ш...,Мария Атанасова,2020-09-14 18:09,3 518,NaN,0.0,['айтос'],6






## Train/Test split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], 
                                                    df['topic_code'], 
                                                    test_size=0.30, 
                                                    random_state=8)

In [15]:
# X
with open('Pickles/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
with open('Pickles/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
    
# Y   
with open('Pickles/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
with open('Pickles/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)

In [16]:
print("Test set size: " + str(y_test.shape))
print("Train set size: " + str(y_train.shape))

Test set size: (6348,)
Train set size: (14811,)


# TD-IDF text reprezentation

There is alot of various options:

TF-IDF Vectors as features
Word Embeddings as features
Count Vectors as features


We will use TF-IDF Vectors as features.


In [17]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 1000



tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(14811, 1000)
(6348, 1000)


In [18]:
for topic_name, topic_code in topic_codes.items():
    chi2_feature = chi2(features_train, labels_train == topic_code)
    indices = np.argsort(chi2_feature[0])
    feature_names = np.array(tfidf.get_feature_names_out())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(topic_name))
    print(". Most correlated unigrams:\n  . {}".format('\n  . '.join(unigrams[-5:])))
    print(". Most correlated bigrams:\n  . {}".format('\n  . '.join(bigrams[-2:])))
    print("")

# 'Бизнес' category:
. Most correlated unigrams:
  . министър
  . лв
  . сто
  . газ
  . туризма
. Most correlated bigrams:
  . публикация споделена
  . млн лв

# 'Имоти' category:
. Most correlated unigrams:
  . жилищата
  . eur
  . жилища
  . квм
  . имоти
. Most correlated bigrams:
  . млн лв
  . недвижими имоти

# 'Технологии' category:
. Most correlated unigrams:
  . технология
  . tesla
  . устройства
  . apple
  . iphone
. Most correlated bigrams:
  . mercedes benz
  . превозни средства

# 'Култура' category:
. Most correlated unigrams:
  . албум
  . филмът
  . поредицата
  . филма
  . филм
. Most correlated bigrams:
  . конски сили
  . най добър

# 'Спорт' category:
. Most correlated unigrams:
  . левски
  . лига
  . клуба
  . мача
  . цска
. Most correlated bigrams:
  . публикация споделена
  . ата минута

# 'Любопитно' category:
. Most correlated unigrams:
  . post
  . певицата
  . бъдете
  . имате
  . пише
. Most correlated bigrams:
  . shared by
  . post shared

# 'Крими' c

In [19]:
tfidf.get_feature_names_out()
unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
bigrams = [v for v in feature_names if len(v.split(' ')) == 2]

print("Unigram count: " + str(len(unigrams)))
print("Bigram count: " + str(len(bigrams)))

Unigram count: 967
Bigram count: 33


As we can see, the unigrams fits well to the topics. However, bigrams do not fit well. There are only 34. This means the unigrams have more correlation with the topics than the bigrams, and only a few bigrams will be considered

In [20]:
# X
with open('Pickles/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
with open('Pickles/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
    
# Y   
with open('Pickles/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
with open('Pickles/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)


# FEATURES
with open('Pickles/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

with open('Pickles/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)


# LABELS
with open('Pickles/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

with open('Pickles/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)

    
# DF   
with open('Pickles/df.pickle', 'wb') as output:
    pickle.dump(df, output) 


# TF-IDF
with open('Pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)